# Basic statistics su `WineQT.csv` duomenimis

Bazinė statistinė analizė su `pandas`, `numpy`, `matplotlib` ir `scipy.stats`:
- duomenų peržiūra ir santrauka
- skirstiniai (histogramos), ryšiai (scatterplot)
- koreliacijos ir koreliacijų matrica
- outlier (išskirčių) aptikimas
- normalumo testai: **Shapiro-Wilk**, **D’Agostino K² (`normaltest`)**, **Anderson-Darling**
- koreliacijos testai: **Pearson**, **Spearman**, **Kendall** 


# Dataset

https://www.kaggle.com/datasets/yasserh/wine-quality-dataset

In [1]:
# Bibliotekos (standartinis rinkinys)
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import shapiro, normaltest, pearsonr, spearmanr, kendalltau, anderson

# Patogesniam vaizdų rodymui
plt.rcParams["figure.figsize"] = (8, 4)


## 1) Duomenų įkėlimas



In [2]:
df = pd.read_csv(r"....WineQT.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '....WineQT.csv'

## 2) Greita peržiūra ir pagrindinė santrauka

In [ ]:
# Struktūra ir pirmos eilutės
display(df.head())
print("Eilučių skaičius:", df.shape[0])
print("Stulpelių skaičius:", df.shape[1])


In [ ]:
# Trūkstamos reikšmės (
df.isna().sum()


In [ ]:
# Tipai ir bazinė statistika
df.info()


In [ ]:
# Skaitinių stulpelių statistika
# include='all' dažnai sukuria daug informacijos, bet pradedantiesiems pakanka skaitinių
df.describe().T.round()


## 3) Skirstiniai: histogramos

In [ ]:
# Pasirenkami keli stulpeliai vizualizacijoms (paprasta ir aišku)
cols = ["alcohol", "pH", "density", "residual sugar", "chlorides", "quality"]

for c in cols:
    plt.figure()
    plt.hist(df[c], bins=30)
    plt.title(f"Histograma: {c}")
    plt.xlabel(c)
    plt.ylabel("Dažnis")
    plt.show()


Geroji praktika:
- pradėti nuo kelių svarbiausių kintamųjų, o ne iškart braižyti visus 12–15
- naudoti aiškius pavadinimus ir ašių etiketes
- lyginti histogramos formą su realia prasme (pvz., `quality` yra diskretus įvertinimas)

Dažna klaida: histogramos interpretavimas kaip „normalus skirstinys“ vien dėl to, kad forma „panaši į varpą“.
Normalumą tiksliau įvertina ir vizualūs metodai (Q-Q), ir testai (toliau).


## 4) Ryšiai tarp kintamųjų: scatterplot

In [ ]:
# Paprastas ir dažnas pavyzdys: alkoholis vs kokybė
plt.figure()
plt.scatter(df["alcohol"], df["quality"], alpha=0.6)
plt.title("Scatterplot: alcohol vs quality")
plt.xlabel("alcohol")
plt.ylabel("quality")
plt.show()


In [ ]:
#reggresion line

# Duomenys
x = df["alcohol"]
y = df["quality"]

# 1) Pearson koreliacija
r, p_value = pearsonr(x, y)

# 2) Linijinė regresija (y = a*x + b)
coef = np.polyfit(x, y, 1)
reg_line = np.poly1d(coef)

# 3) R² (paaiškintos dispersijos dalis)
y_pred = reg_line(x)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - ss_res / ss_tot

# 4) Vizualizacija
plt.figure(figsize=(8, 5))
plt.scatter(x, y, alpha=0.6)
plt.plot(x, reg_line(x))

plt.title("Alcohol vs Quality su regresijos linija")
plt.xlabel("alcohol")
plt.ylabel("quality")

# Tekstas grafike
plt.text(
    x.min(),
    y.max(),
    f"Pearson r = {r:.2f}\nR² = {r_squared:.2f}",
    verticalalignment="top"
)

plt.tight_layout()
plt.show()


In [ ]:
# Kitas dažnas ryšys: volatile acidity vs quality
plt.figure()
plt.scatter(df["volatile acidity"], df["quality"], alpha=0.6)
plt.title("Scatterplot: volatile acidity vs quality")
plt.xlabel("volatile acidity")
plt.ylabel("quality")
plt.show()


In [ ]:
# regression line

# Duomenys
x = df["volatile acidity"]
y = df["quality"]

# 1) Pearson koreliacija
r, p_value = pearsonr(x, y)

# 2) Linijinė regresija (y = a*x + b)
coef = np.polyfit(x, y, 1)
reg_line = np.poly1d(coef)

# 3) R²
y_pred = reg_line(x)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - ss_res / ss_tot

# 4) Vizualizacija
plt.figure(figsize=(8, 5))
plt.scatter(x, y, alpha=0.6)
plt.plot(x, reg_line(x))

plt.title("Volatile acidity vs Quality su regresijos linija")
plt.xlabel("volatile acidity")
plt.ylabel("quality")

plt.text(
    x.min(),
    y.max(),
    f"Pearson r = {r:.2f}\nR² = {r_squared:.2f}",
    verticalalignment="top"
)

plt.tight_layout()
plt.show()


## 5) Koreliacijos

In [ ]:
# Pearson koreliacijų matrica (greita santrauka)
corr_pearson = df.drop(columns=["Id"], errors="ignore").corr(method="pearson")
corr_pearson


In [ ]:
# Koreliacijų matricos vizualizacija (be seaborn)
plt.figure(figsize=(10, 8))
plt.imshow(corr_pearson, aspect="auto")
plt.title("Koreliacijų matrica (Pearson)")
plt.xticks(range(len(corr_pearson.columns)), corr_pearson.columns, rotation=90)
plt.yticks(range(len(corr_pearson.columns)), corr_pearson.columns)
plt.colorbar()
plt.tight_layout()
plt.show()


In [ ]:

# Koreliacijų matricos vizualizacija (su seaborn)

plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_pearson,
    annot=True,        # rodo koreliacijos reikšmes
    fmt=".2f",         # 2 skaitmenys po kablelio
    cmap="coolwarm",   # spalvų skalė
    center=0,          # 0 – neutralus taškas
    square=True
)

plt.title("Koreliacijų matrica (Pearson)")
plt.tight_layout()
plt.show()


### 5.2 Koreliacijos testai dviem kintamiesiems

In [ ]:
# Funkcija, kuri grąžina Pearson, Spearman ir Kendall testus
def corr_tests(x, y):
    x = np.asarray(x)
    y = np.asarray(y)

    # Pearson
    r_p, p_p = pearsonr(x, y)
    # Spearman
    r_s, p_s = spearmanr(x, y)
    # Kendall
    r_k, p_k = kendalltau(x, y)

    return pd.DataFrame(
        {
            "method": ["pearson", "spearman", "kendall"],
            "statistic": [r_p, r_s, r_k],
            "p_value": [p_p, p_s, p_k],
        }
    )

# Pavyzdys: alcohol ir quality
corr_tests(df["alcohol"], df["quality"])


## 6) Outlier (išskirtys)

In [ ]:
# IQR metodas (paprastas ir dažnas)
def iqr_outliers(series, k=1.5):
    s = series.dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    mask = (s < lower) | (s > upper)
    return mask.sum(), lower, upper

outlier_summary = []
numeric_cols = df.select_dtypes(include="number").columns.tolist()
if "Id" in numeric_cols:
    numeric_cols.remove("Id")

for c in numeric_cols:
    n_out, lo, hi = iqr_outliers(df[c])
    outlier_summary.append([c, n_out, lo, hi])

pd.DataFrame(outlier_summary, columns=["column", "outliers_iqr_count", "lower_bound", "upper_bound"])   .sort_values("outliers_iqr_count", ascending=False)


In [ ]:
# Boxplot keliems kintamiesiems (aišku, kas turi ilgus "ūsus") 

cols_box = ["alcohol", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide"]

plt.figure(figsize=(10, 4))
plt.boxplot(
    [df[c].dropna() for c in cols_box],
    labels=cols_box,
    vert=True
)
plt.title("Boxplot: pasirinkti kintamieji")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## 7) Normalumo įvertinimas

In [ ]:
# Q-Q grafikas (vizualus normalumo patikrinimas)
def qq_plot(series, title):
    x = series.dropna().values
    plt.figure()
    stats.probplot(x, dist="norm", plot=plt)
    plt.title(title)
    plt.show()

qq_plot(df["alcohol"], "Q-Q: alcohol")
qq_plot(df["residual sugar"], "Q-Q: residual sugar")


### 7.1 Shapiro-Wilk testas

In [ ]:
# Shapiro rekomenduojamas mažesnėms imtims, bet naudojamas ir edukacijai.
# Jei N labai didelis, galima paimti atsitiktinį mėginį, kad testas būtų stabilesnis ir greitesnis.

def shapiro_test(series, sample_n=5000, random_state=42):
    x = series.dropna()
    if len(x) > sample_n:
        x = x.sample(sample_n, random_state=random_state)
    stat, p = shapiro(x)
    return stat, p, len(x)

for c in ["alcohol", "pH", "density"]:
    stat, p, n = shapiro_test(df[c])
    print(f"{c:>10} | n={n:>5} | Shapiro stat={stat:.4f} | p={p:.4g}")


### 7.2 D’Agostino K² (`normaltest`)

In [ ]:
# normaltest reikalauja pakankamai didelės imties (paprastai N >= 8)
def dagostino_test(series):
    x = series.dropna().values
    stat, p = normaltest(x)
    return stat, p, len(x)

for c in ["alcohol", "pH", "density"]:
    stat, p, n = dagostino_test(df[c])
    print(f"{c:>10} | n={n:>5} | K2 stat={stat:.4f} | p={p:.4g}")


### 7.3 Anderson-Darling testas

In [ ]:
# Anderson grąžina test statistic ir kritines reikšmes skirtingiems reikšmingumo lygiams
def anderson_test(series):
    x = series.dropna().values
    res = anderson(x, dist="norm")
    out = pd.DataFrame(
        {"significance_level_%": res.significance_level, "critical_value": res.critical_values}
    )
    return res.statistic, out, len(x)

stat, table, n = anderson_test(df["alcohol"])
print(f"alcohol | n={n} | Anderson stat={stat:.4f}")
display(table)
